In [ ]:
import pandas as pd
import numpy as np
import re
import math
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics, model_selection
import xgboost as xgb
from xgboost import XGBClassifier

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Datasets for Q1
sales = pd.read_csv('algorun-2021-data/sales.csv')
hierarc = pd.read_csv('algorun-2021-data/product_hierarchy.csv')
store = pd.read_csv('algorun-2021-data/store_cities.csv')

In [ ]:
sales.head()

In [ ]:
hierarc.head()

In [ ]:
sales.describe(include="all")

In [ ]:
# Drop promo_discount_type_2, promo_discount_2, promo_bin_2, promo_bin_1
sales.isna().sum().sort_values(ascending=False)/len(sales)

In [ ]:
sales.drop(["promo_discount_type_2", "promo_discount_2", "promo_bin_2", "promo_bin_1"],axis=1, inplace=True)


In [ ]:
sales = sales.dropna(subset=['sales'])

In [ ]:
# Merge product and sales dataframes
df = pd.merge(sales, hierarc[['product_id', 'hierarchy1_id', 'hierarchy2_id', 'hierarchy3_id', 'hierarchy4_id']], left_on='product_id', right_on='product_id')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include="all")

In [ ]:
df.isna().sum().sort_values(ascending=False)/len(df)

In [ ]:
# It will convert columns that have string dtypes to pandas category type
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()
            
df_categorized = df.copy()
train_cats(df_categorized)

In [ ]:
# Missing of the entry can also be a valuable information
# So we will create a column that is False when value is missing
# We encoded missingness in categorical columns so we will just create _na columns for numerical types
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())
        
# We will have codes starting from 0 (for missing)
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes+1
          
def proc_df(df):
    
#     y = df[y_fld].values
#     df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        
    for n, c in df.items():
        numericalize(df, c, n)
    
#     y = df[y_fld].values
#     df.drop([y_fld], axis = 1, inplace = True)
    
#     res = [df, y]
    
    return df

In [ ]:
df_visual = df_categorized.copy()


In [ ]:
df_visual = proc_df(df_visual)

In [ ]:
# En yüksek korelasyon date ve price ile arasında var
plt.figure(figsize=(12,10))
sns.heatmap(df_visual.corr())

In [ ]:
#dt_name is the name of the column that is of type datepart
def add_datepart(df, dt_name, drop=True, time=False):
    "Creates new columns from our datetime column"
    
    
    dt_column = df[dt_name]
    column_dtype = dt_column.dtype
    

    targ_name = re.sub('[Dd]ate$', '', dt_name)
    
    # attributes are normally in lower case but we wrote this way because we will use it in columns' name too
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
        
    
    #Sorry curse of dimensionality, maybe another time
    for a in attr: 
        df[targ_name + a] = getattr(dt_column.dt, a.lower())
        
    # how much time passed, we will divide by 10^9 because it is in the nanosecond format
    df[targ_name + 'Elapsed'] = dt_column.astype(np.int64) // 10 ** 9
    
    if drop: 
        df.drop(dt_name, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df_new = df.copy()
df_new.date = pd.to_datetime(df_new.date)
add_datepart(df_new, 'date')

In [ ]:
df_new.head()

In [ ]:
df_sales = pd.DataFrame(df.groupby(by=["product_id","hierarchy4_id","date"],as_index=False)["sales"].sum())
df_sales.head(20)

In [ ]:
df_sales_2017 = df_sales[(df_sales["date"]>="2017-01-01") & (df_sales["date"]<="2017-12-31")]
df_sales_2018 = df_sales[(df_sales["date"]>="2018-01-01") & (df_sales["date"]<="2018-12-31")]
df_sales_2017.head(20)

In [ ]:
df_sales_2017.groupby(by="hierarchy4_id").nunique().head(30)

In [ ]:
#2018 yılı için yıl boyunca aynı hiyerarşi 4 id'sine sahip ürünler benzer satış yükseliş ve artışı göstermiş.
# hierarchy4_id=H00000405 
plt.figure(figsize=(12,10))
plt.plot(df_sales_2017[df_sales_2017["product_id"]=='P0110']["date"], df_sales_2017[df_sales_2017["product_id"]=='P0110']["sales"], "b")
plt.plot(df_sales_2017[df_sales_2017["product_id"]=='P0249']["date"], df_sales_2017[df_sales_2017["product_id"]=='P0249']["sales"], "r")
plt.plot(df_sales_2017[df_sales_2017["product_id"]=='P0265']["date"], df_sales_2017[df_sales_2017["product_id"]=='P0265']["sales"], "m")
plt.plot(df_sales_2017[df_sales_2017["product_id"]=='P0427']["date"], df_sales_2017[df_sales_2017["product_id"]=='P0427']["sales"], "y")

In [ ]:
#2018 yılı için yıl boyunca aynı hiyerarşi 4 id'sine sahip ürünler benzer satış yükseliş ve artışı göstermiş.
# hierarchy4_id=H00000405 
plt.figure(figsize=(12,10))
plt.plot(df_sales_2018[df_sales_2018["product_id"]=='P0116']["date"], df_sales_2018[df_sales_2018["product_id"]=='P0116']["sales"], "b")
plt.plot(df_sales_2018[df_sales_2018["product_id"]=='P0198']["date"], df_sales_2018[df_sales_2018["product_id"]=='P0198']["sales"], "r")
plt.plot(df_sales_2018[df_sales_2018["product_id"]=='P0590']["date"], df_sales_2018[df_sales_2018["product_id"]=='P0590']["sales"], "m")

In [ ]:
## Sample
def get_sample(df,n):
    df.reset_index(inplace=True, drop=True)
    idxs = np.random.permutation(len(df))[:n]
    return idxs, df.iloc[idxs].copy()

# Creating validation set
# It will split our data set to have length n train and len(df) - n validation set
def split_train_val(df,n): 
    return df[:n].copy(), df[n:].copy()

# Creating validation set
# It will split our data set to have length n train and len(df) - n validation set
def split_train_val(df,n): 
    return df[:n].copy(), df[n:].copy()

def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

def print_score(m):  
    print(f"RMSE of train set {rmse(m.predict(X_train), y_train)}")
    print(f"RMSE of validation set {rmse(m.predict(X_valid), y_valid)}")
    print(f"R^2 of train set {m.score(X_train, y_train)}")
    print(f"R^2 of validation set {m.score(X_valid, y_valid)}")
    

In [ ]:
train_cats(df_new)

In [ ]:
df_new = df_new.drop(["product_id"], axis = 1)
df_new.head()

In [ ]:
# Missing of the entry can also be a valuable information
# So we will create a column that is False when value is missing
# We encoded missingness in categorical columns so we will just create _na columns for numerical types
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())
        
# We will have codes starting from 0 (for missing)
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes+1
          
def proc_df(df, y_fld):
    
#     y = df[y_fld].values
#     df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        
    for n, c in df.items():
        numericalize(df, c, n)
    
    y = df[y_fld].values
    df.drop([y_fld], axis = 1, inplace = True)
    
    res = [df, y]
    
    return res

In [ ]:
df_new.info()

In [ ]:
df_hier1, y_hier1 = proc_df(df_new.drop(columns=['hierarchy3_id', 'hierarchy1_id', 'hierarchy2_id']), 'hierarchy4_id')

In [ ]:
df_hier1.shape, y_hier1.shape

In [ ]:
df_hier1.head()

In [ ]:
X_sample, nc, y_sample,na = model_selection.train_test_split(df_hier1, y_hier1, train_size = 0.005, random_state = 42, stratify=y_hier1)
X_sample.shape, y_sample.shape


In [ ]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X_sample, y_sample, train_size = 0.8, random_state = 42, stratify=y_sample)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
X_train_2, X_test, y_train_2, y_test = model_selection.train_test_split(X_train, y_train, train_size = 0.9, random_state = 42, stratify=y_train)
X_train_2.shape, X_test.shape, y_train_2.shape, y_test.shape

In [ ]:
m = RandomForestRegressor(n_estimators=100, max_depth=3, bootstrap=True, n_jobs=-1)
%time m.fit(X_train_2, y_train_2)
print_score(m)

In [ ]:
alg = XGBClassifier(learning_rate=0.4, n_estimators=220, max_depth=5,
                        min_child_weight=3, subsample=0.6,
                        objective='binary:logistic', nthread=4, seed=27)
%time alg.fit(X_train_2, y_train_2)
print_score(alg)

In [ ]:
def rf_feat_importance(alg, df):
    return pd.DataFrame({'columns':df.columns, 'importance':alg.feature_importances_}
                       ).sort_values('importance', ascending=False)

In [ ]:
rf_feat_importance(alg, X_train_2)